In [ ]:
# !sed -i 's/transformers.generation_utils/transformers.generation/' /usr/local/lib/python3.12/dist-packages/openprompt/pipeline_base.py

In [ ]:
# !pip install transformers==4.24.0

In [ ]:
import os
import shutil

# Get user profile path
user_profile = os.environ["USERPROFILE"]

# Paths to Hugging Face cached models
cached_models = [
    os.path.join(user_profile, r".cache\huggingface\hub\models--bert-base-multilingual-cased"),
    os.path.join(user_profile, r".cache\huggingface\hub\models--xlm-roberta-base")
]

# Remove cached models if they exist
for path in cached_models:
    if os.path.exists(path):
        shutil.rmtree(path)
        print(f"Removed cache: {path}")
    else:
        print(f"No cache found at: {path}")


In [1]:
# Set device
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
# ==============================
# Imports
# ==============================
import torch
import pandas as pd
import random
import numpy as np
from collections import Counter
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
from openprompt.prompts import ManualTemplate, ManualVerbalizer
from openprompt.prompts import SoftVerbalizer
from openprompt.prompts import AutomaticVerbalizer
from openprompt.data_utils import InputExample
from openprompt.plms import load_plm
from openprompt import PromptForClassification, PromptDataLoader
from torch.optim import AdamW
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader, Sampler
from collections import defaultdict

C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# ========================================
# Check CUDA
# ========================================
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))
    print("CUDA version:", torch.version.cuda)
    print("GPU count:", torch.cuda.device_count())

# ========================================
# Seeds for reproducibility
# ========================================
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

CUDA available: True
GPU name: NVIDIA GeForce RTX 4080 SUPER
CUDA version: 12.1
GPU count: 1


In [4]:
# ==============================
# Balanced Batch Sampler
# ==============================
class BalancedBatchSampler(Sampler):
    def __init__(self, dataset, batch_size):
        """
        dataset: list of InputExample
        batch_size: total batch size (must be divisible by number of classes)
        """
        self.dataset = dataset
        self.labels = [ex.label for ex in dataset]
        self.classes = list(sorted(set(self.labels)))
        self.num_classes = len(self.classes)
        assert batch_size % self.num_classes == 0, "Batch size must be divisible by number of classes"
        self.batch_size_per_class = batch_size // self.num_classes

    def __iter__(self):
        class_indices = {c: np.where(np.array(self.labels) == c)[0].tolist() for c in self.classes}
        for c in self.classes:
            np.random.shuffle(class_indices[c])

        num_batches = min(len(class_indices[c]) // self.batch_size_per_class for c in self.classes)

        for i in range(num_batches):
            batch = []
            for c in self.classes:
                start = i * self.batch_size_per_class
                end = start + self.batch_size_per_class
                batch.extend(class_indices[c][start:end])
            np.random.shuffle(batch)
            yield batch

    def __len__(self):
        return min(len(np.where(np.array(self.labels) == c)[0]) // self.batch_size_per_class for c in self.classes)

In [5]:

# Step 1: Training data (16-shot) ha,m and spam
train_dataset = [

    InputExample(guid=0, text_a="یہاں کے ڈاکٹر بہت باخبر ہیں۔", label=0),
    InputExample(guid=1, text_a="یہ درس گاہ میں جدید سہولیات موجود ہیں۔", label=0),
    InputExample(guid=2, text_a="ورزش جسمانی اور ذہنی صحت کو بہتر بناتی ہے۔", label=0),
    InputExample(guid=3, text_a="یہ ایک قابل ذکر پیشرفت ہے۔", label=0),
    InputExample(guid=4, text_a="یہ پالیسی آبادی کو مستحکم کرنے میں مددگار ہے۔", label=0),
    InputExample(guid=5, text_a="آج موسم بہت خوشگوار ہے۔", label=0),
    InputExample(guid=6, text_a="اس شعبے میں تحقیق کی بہت گنجائش ہے۔", label=0),
    InputExample(guid=7, text_a="میں آپ کے کام سے بہت مطمئن ہوں۔", label=0),
    InputExample(guid=8, text_a="یہ ایک نہایت ہی خوشگوار تجربہ تھا۔", label=0),
    InputExample(guid=9, text_a="یہ گارڈن بہت خوبصورت ہے۔", label=0),
    InputExample(guid=10, text_a="مجھے یہ گانا بہت اچھا لگا۔", label=0),
    InputExample(guid=11, text_a="آپ کی خدمات کی تعریف کی جاتی ہے۔", label=0),
    InputExample(guid=12, text_a="یہ پروجیکٹ بہت کامیاب رہا ہے۔", label=0),
    InputExample(guid=13, text_a="اساتذہ کا رویہ بہت دوستانہ ہے۔", label=0),
    InputExample(guid=14, text_a="اسکول کا ماحول بہت تعلیمی اور حوصلہ افزا ہے۔", label=0),
    InputExample(guid=15, text_a="مجھے روزانہ ورزش کرنی چاہیے۔", label=0),

    
    InputExample(guid=16, text_a="اس علاقے میں آبادی بہت زیادہ ہے۔", label=1),
    InputExample(guid=17, text_a="یہاں بجلی کی بندش کا مسئلہ بہت عام ہے۔", label=1),
    InputExample(guid=18, text_a="مجھے شدید کھانسی ہو رہی ہے۔", label=1),
    InputExample(guid=19, text_a="مجھے بخار اور گلے میں درد کی شکایت ہے۔", label=1),
    InputExample(guid=20, text_a="غریب مریضوں کے لیے مفت ادویات کا بندوبست کیا جائے۔", label=1),
    InputExample(guid=21, text_a="ڈاکٹر نے غلط تشخیص کی وجہ سے پریشانی میں مبتلا کیا۔", label=1),
    InputExample(guid=22, text_a="یہ میڈیکل رپورٹ غلط لکھی گئی ہے، مجھے شکایت ہے۔", label=1),
    InputExample(guid=23, text_a="حکومتی پالیسیاں صحت کے معیار کو بہتر بنانے میں ناکام رہی ہیں۔", label=1),
    InputExample(guid=24, text_a="میں نے کل شام سے کھانا نہیں کھایا، مجھے بہت کمزوری محسوس ہو رہی ہے۔", label=1),
    InputExample(guid=25, text_a="حکومت آبادی کنٹرول کے لیے سنجیدہ اقدامات نہیں کر رہی۔", label=1),
    InputExample(guid=26, text_a="میں اپنی صحت کے بارے میں فکر مند ہوں۔", label=1),
    InputExample(guid=27, text_a="میں علاج کے دوران ڈاکٹر کے رویے سے ناخوش ہوں۔", label=1),
    InputExample(guid=28, text_a="بڑھتی ہوئی آبادی کا مطلب ہے وسائل پر زیادہ دباؤ۔", label=1),
    InputExample(guid=29, text_a="سکولوں میں کھیل کے میدان موجود نہیں ہیں۔", label=1),
    InputExample(guid=30, text_a="میں اسکول میں ہونے والے تشدد کے بارے میں شکایت کرنا چاہتا ہوں۔", label=1),
    InputExample(guid=31, text_a="اس اسپتال میں علاج بہت مہنگا ہے، غریب مریضوں کا کیا بنے گا؟", label=1),


    InputExample(guid=32, text_a="قومی سطح پر آبادی کنٹرول کے اقدامات پر زور دیا جائے۔", label=2),
    InputExample(guid=33, text_a="موسمی بیماریوں سے بچاؤ کے لیے احتیاطی تدابیر اختیار کرنی چاہییں۔", label=2),
    InputExample(guid=34, text_a="کیا آپ ہمیں آن لائن رجسٹریشن کے بارے میں بتا سکتے ہیں؟", label=2),
    InputExample(guid=35, text_a="میں اس کے بارے میں مزید جاننا چاہتا ہوں۔", label=2),
    InputExample(guid=36, text_a="میں اپنے والد کے لیے ادویات کا نسخہ دوبارہ لکھوانا چاہتا ہوں۔", label=2),
    InputExample(guid=37, text_a="کیا آپ مجھے قریب ترین ریلوے سٹیشن کا پتہ بتا سکتے ہیں؟", label=2),
    InputExample(guid=38, text_a="کیا آپ یہ سامان اٹھا سکتے ہیں؟", label=2),
    InputExample(guid=39, text_a="میں اس آبادیاتی رپورٹ کا خلاصہ جاننا چاہتا ہوں۔", label=2),
    InputExample(guid=40, text_a="براہ کرم مجھے ایک نیا فون دلوا دو۔", label=2),
    InputExample(guid=41, text_a="مجھے حصول تعلیم کے مواقع کے بارے میں جاننا ہے۔", label=2),
    InputExample(guid=42, text_a="مجھے حصول تعلیم کے مواقع کے بارے میں جاننا ہے۔", label=2),
    InputExample(guid=43, text_a="کیا اس سال اسکالر شپ کے لیے درخواستیں قبول کی جا رہی ہیں؟", label=2),
    InputExample(guid=44, text_a="مجھے یہ جاننا ہے کہ سب سے زیادہ آبادی والا شہر کون سا ہے۔", label=2),
    InputExample(guid=45, text_a="میں نے ایک نیا لیپ ٹاپ خریدا ہے۔", label=2),
    InputExample(guid=46, text_a="مجھے سر درد کی دوا چاہیے، شکریہ۔", label=2),
    InputExample(guid=47, text_a="کیا کالج میں داخلے کی آخری تاریخ میں توسیع ممکن ہے؟", label=2),
    
    
    
]


In [6]:
# ==============================
# Define Classes
# ==============================
classes = ["positive", "negative","neutral"]
label_map = {"positive": 0, "negative": 1, "neutral": 2}

# # Step 1: Use load_plm with 'roberta' to get the correct WrapperClass
_, _, _, WrapperClass = load_plm("roberta", "roberta-base")  # Just to get the wrapper

# # Step 2: Manually load XLM-RoBERTa model/tokenizer
model_name = "xlm-roberta-base"
tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
plm = XLMRobertaForMaskedLM.from_pretrained(model_name)

# ==============================
# Load Pretrained Language Model (mBERT)
# ==============================
# plm, tokenizer, model_config, WrapperClass = load_plm("bert", "bert-base-multilingual-cased")


C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:

# ==============================
# Define Prompt Template (Manual)
# ==============================
template = ManualTemplate( 
    text='{"placeholder":"text_a"} یہ جملہ {"mask"} ہے۔',
    tokenizer=tokenizer,
)

 

templates = [
    ("P1", ManualTemplate(
        text='{"placeholder":"text_a"} یہ جملہ {"mask"} ہے۔',  # Keep as is
        tokenizer=tokenizer,
    )),
    ("P2", ManualTemplate(
        text='دی گئی عبارت: {"placeholder":"text_a"} کا مواد اور پیغام {"mask"} ہے۔',  # generalized for neg/pos
        tokenizer=tokenizer,
    )),
    ("P3", ManualTemplate(
        text='اس عبارت کا تجزیہ کریں: {"placeholder":"text_a"} اس کا احساس یا موڈ {"mask"} ہے۔',  # focuses on sentiment/intent
        tokenizer=tokenizer,
    )),
    ("P4", ManualTemplate(
        text='جملہ: {"placeholder":"text_a"} ہمیں بتاتا ہے کہ یہ مواد {"mask"} ہے۔',  # clear statement aligned with label
        tokenizer=tokenizer,
    )),
    ("P5", ManualTemplate(
        text='{"placeholder":"text_a"} اس عبارت کا مطلب {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),
    ("P6", ManualTemplate(
        text='{"placeholder":"text_a"} اس معاملے میں حتمی رائے {"mask"}',
        tokenizer=tokenizer,
    )),
    ("P7", ManualTemplate(
        text='{"placeholder":"text_a"} اس مواد کی تشریح {"mask"}',
        tokenizer=tokenizer,
    )),
    ("P8", ManualTemplate(
        text='{"placeholder":"text_a"} اس حوالے سے فیصلہ {"mask"}',
        tokenizer=tokenizer,
    )),
    ("P9", ManualTemplate(
        text='{"placeholder":"text_a"} اس متن کی درجہ بندی {"mask"}',
        tokenizer=tokenizer,
    )),
    ("P10", ManualTemplate(
        text='{"placeholder":"text_a"} اس اظہار کا نتیجہ {"mask"}',
        tokenizer=tokenizer,
    )), 
]

 

In [8]:
# ==============================
# Define Verbalizer (Manual)
# ==============================
verbalizer = ManualVerbalizer(
    classes=classes,
    label_words = {
    "positive": ["مثبت", "خوشگوار"],
    "negative": ["منفی", "مسئلہ"],
    "neutral":  ["غیرجانبدار", "معلوماتی"],
        
    },
    tokenizer=tokenizer,
)

In [9]:
# ==============================
# Create Prompt Model
# ==============================
prompt_model = PromptForClassification(
    template=template,
    plm=plm,
    verbalizer=verbalizer
)

# ==============================
# Training loop with BalancedBatchSampler + random template switching
# ==============================
T = 20   # epochs
K = 1    # steps per prompt
batch_size = 12





prompt_model.train()
optimizer = AdamW(prompt_model.parameters(), lr=1e-5)
all_epoch_patterns = {}

for epoch in range(T):
    print(f"\n🟦 Epoch {epoch+1}/{T}")

    # Random initial template
    prompt_name, current_template = random.choice(templates)
    epoch_pattern = []

    # Create PromptDataLoader with BalancedBatchSampler
    sampler = BalancedBatchSampler(train_dataset, batch_size=batch_size)
    train_loader = PromptDataLoader(
        dataset=train_dataset,
        tokenizer=tokenizer,
        template=current_template,
        tokenizer_wrapper_class=WrapperClass,
        max_seq_length=128,
        batch_size=batch_size,
        batch_sampler=sampler,
        shuffle=False  # shuffle is ignored when batch_sampler is used
    )

    step_counter = 0
    epoch_loss = 0.0

    for batch in train_loader:
        # Move batch to device
        # batch = {k: v.to(device) for k, v in batch.items()}
    
        optimizer.zero_grad()
        logits = prompt_model(batch)
        loss = torch.nn.CrossEntropyLoss()(logits, batch['label'])
        
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_pattern.append(prompt_name)

        step_counter += 1

        # Switch template every K steps
        if step_counter % K == 0:
            prompt_name, current_template = random.choice(templates)

            # Rebuild PromptDataLoader with new template but same sampler
            train_loader = PromptDataLoader(
                dataset=train_dataset,
                tokenizer=tokenizer,
                template=current_template,
                tokenizer_wrapper_class=WrapperClass,
                max_seq_length=128,
                batch_size=batch_size,
                batch_sampler=sampler,
                shuffle=False
            )

    all_epoch_patterns[f"epoch_{epoch+1}"] = epoch_pattern
    print(f"Epoch {epoch+1} Loss: {epoch_loss:.4f}")
    print(f"Prompt pattern: {epoch_pattern}")



🟦 Epoch 1/20


tokenizing: 48it [00:00, 2590.81it/s]
tokenizing: 48it [00:00, 2450.06it/s]
tokenizing: 48it [00:00, 2521.50it/s]
tokenizing: 48it [00:00, 2128.12it/s]
tokenizing: 48it [00:00, 2036.73it/s]


Epoch 1 Loss: 9.5464
Prompt pattern: ['P2', 'P1', 'P5', 'P4']

🟦 Epoch 2/20


tokenizing: 48it [00:00, 2037.84it/s]
tokenizing: 48it [00:00, 1994.36it/s]
tokenizing: 48it [00:00, 2739.03it/s]
tokenizing: 48it [00:00, 2038.73it/s]
tokenizing: 48it [00:00, 2590.88it/s]


Epoch 2 Loss: 4.6105
Prompt pattern: ['P3', 'P2', 'P9', 'P2']

🟦 Epoch 3/20


...

Epoch 18 Loss: 0.2822
Prompt pattern: ['P1', 'P6', 'P7', 'P5']

🟦 Epoch 19/20


tokenizing: 48it [00:00, 2906.66it/s]
tokenizing: 48it [00:00, 4170.15it/s]
tokenizing: 48it [00:00, 4170.24it/s]
tokenizing: 48it [00:00, 2853.35it/s]
tokenizing: 48it [00:00, 4360.64it/s]


Epoch 19 Loss: 0.2480
Prompt pattern: ['P4', 'P10', 'P6', 'P4']

🟦 Epoch 20/20


tokenizing: 48it [00:00, 4171.10it/s]
tokenizing: 48it [00:00, 3996.40it/s]
tokenizing: 48it [00:00, 3092.95it/s]
tokenizing: 48it [00:00, 3551.74it/s]
tokenizing: 48it [00:00, 2997.40it/s]

Epoch 20 Loss: 0.1991
Prompt pattern: ['P7', 'P8', 'P3', 'P5']


In [10]:
df = pd.read_csv(r"C:\Users\stdFurqan\Desktop\paft\multi\urdu_multidomain_dev_split_no_request.csv")
eval_dataset = [
    InputExample(guid=i, text_a=row['text'], label=label_map[row['sentiment']])
    for i, row in df.iterrows()
]

eval_loader = PromptDataLoader(
    dataset=eval_dataset,
    tokenizer=tokenizer,
    template=template,
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=128,
    batch_size=8,
    shuffle=False
)

# ==============================
# Evaluate Model
# ==============================
prompt_model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in eval_loader:
        # Move all tensors in the batch to the device (CPU/GPU)
        # batch = {k: v.to(device) for k, v in batch.items()}

        logits = prompt_model(batch)
        preds = torch.argmax(logits, dim=-1)

        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(batch['label'].cpu().tolist())

# ==============================
# Print Classification Report
# ==============================
print("\n📊 Fine-tuned Classification Report:")
print(classification_report(all_labels, all_preds, target_names=classes, digits=4))

tokenizing: 6923it [00:01, 4757.84it/s]



📊 Fine-tuned Classification Report:
              precision    recall  f1-score   support

    positive     0.8681    0.8380    0.8528      2192
    negative     0.8087    0.8997    0.8517      2053
     neutral     0.8482    0.7991    0.8229      2678

    accuracy                         0.8413      6923
   macro avg     0.8417    0.8456    0.8425      6923
weighted avg     0.8428    0.8413    0.8409      6923



In [ ]:
roberta
📊 Fine-tuned Classification Report:
              precision    recall  f1-score   support

    positive     0.8681    0.8380    0.8528      2192
    negative     0.8087    0.8997    0.8517      2053
     neutral     0.8482    0.7991    0.8229      2678

    accuracy                         0.8413      6923
   macro avg     0.8417    0.8456    0.8425      6923
weighted avg     0.8428    0.8413    0.8409      6923

In [ ]:
mbert

📊 Fine-tuned Classification Report:
              precision    recall  f1-score   support

    positive     0.5702    0.6711    0.6165      2192
    negative     0.5886    0.3561    0.4437      2053
     neutral     0.7346    0.8506    0.7884      2678

    accuracy                         0.6471      6923
   macro avg     0.6311    0.6259    0.6162      6923
weighted avg     0.6392    0.6471    0.6317      6923